In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train400/test_398.png
/kaggle/input/train400/test_032.png
/kaggle/input/train400/test_173.png
/kaggle/input/train400/test_277.png
/kaggle/input/train400/test_021.png
/kaggle/input/train400/test_116.png
/kaggle/input/train400/test_360.png
/kaggle/input/train400/test_095.png
/kaggle/input/train400/test_334.png
/kaggle/input/train400/test_102.png
/kaggle/input/train400/test_304.png
/kaggle/input/train400/test_354.png
/kaggle/input/train400/test_127.png
/kaggle/input/train400/test_015.png
/kaggle/input/train400/test_093.png
/kaggle/input/train400/test_126.png
/kaggle/input/train400/test_100.png
/kaggle/input/train400/test_108.png
/kaggle/input/train400/test_014.png
/kaggle/input/train400/test_267.png
/kaggle/input/train400/test_130.png
/kaggle/input/train400/test_166.png
/kaggle/input/train400/test_055.png
/kaggle/input/train400/test_141.png
/kaggle/input/train400/test_275.png
/kaggle/input/train400/test_345.png
/kaggle/input/train400/test_215.png
/kaggle/input/train400/test_

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader

# Define Model

In [5]:
class DnCNN(nn.Module):
    def __init__(self, num_layers, num_filters=64, image_channels=1):
        super(DnCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=num_filters, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        
        layers = []
        for _ in range(num_layers - 2):
            layers.append(nn.Conv2d(in_channels=num_filters, out_channels=num_filters, kernel_size=3, padding=1))
            layers.append(nn.BatchNorm2d(num_filters))
            layers.append(nn.ReLU(inplace=True))
            
        self.middle_layers = nn.Sequential(*layers)
        
        self.final = nn.Conv2d(in_channels=num_filters, out_channels=image_channels, kernel_size=3, padding=1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.middle_layers(x)
        x = self.final(x)
        return x

In [6]:
model = DnCNN(2)

# Create Dataset

In [7]:
from PIL import Image
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

folder_path = '/kaggle/input/train400'

train_y = []
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for file in files:
    file_path = os.path.join(folder_path, file)
    image = Image.open(file_path).convert('L')
    image_tensor = transform(image)
    train_y.append(image_tensor)
    
train_x = []
for image_tensor in train_y:
    noise = torch.randn(image_tensor.shape) * (25/255)
    image_tensor = image_tensor.clone() + noise
    image_tensor = torch.clamp(image_tensor, 0, 1)
    train_x.append(image_tensor)

In [8]:
# print(train_y[:5])
# print()
# print(train_x[:5])

# Create Dataset Class for Dataloader

In [9]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y
        
    def __len__(self):
        return len(self.train_x)
    
    def __getitem__(self, i):
        return self.train_x[i], self.train_y[i]

# Training

In [10]:
training_dataset = ImageDataset(train_x, train_y)
train_dataloader = DataLoader(training_dataset, batch_size=32, shuffle=True)

In [11]:
optimizer = torch.optim.Adam(model.parameters())
MSELoss = torch.nn.MSELoss()

epochs = 10

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    
    for train_x, train_y in train_dataloader:
        residual_output = model(train_x)
        output = train_x - residual_output
        loss = MSELoss(output, train_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    average_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch: {epoch+1} Loss: {average_loss:.4f}")

Epoch: 1 Loss: 0.0143
Epoch: 2 Loss: 0.0090
Epoch: 3 Loss: 0.0061
Epoch: 4 Loss: 0.0037
Epoch: 5 Loss: 0.0031
Epoch: 6 Loss: 0.0028
Epoch: 7 Loss: 0.0027
Epoch: 8 Loss: 0.0026
Epoch: 9 Loss: 0.0025
Epoch: 10 Loss: 0.0025


# Prepare Test Dataset

In [16]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])
folder_path = '/kaggle/input/bsd68'

test_y = []
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for file in files:
    file_path = os.path.join(folder_path, file)
    image = Image.open(file_path).convert('L')
    image_tensor = transform(image)
    test_y.append(image_tensor)
    
test_x = []
for image_tensor in test_y:
    noise = torch.randn(image_tensor.shape) * (25/255)
    image_tensor = image_tensor.clone() + noise
    image_tensor = torch.clamp(image_tensor, 0, 1)
    test_x.append(image_tensor)

In [ ]:
testing_dataset = ImageDataset(test_x, test_y)
test_dataloader = DataLoader(testing_dataset, batch_size=32, shuffle=False)

In [18]:
# test = torch.stack(test_x)
# test.shape

torch.Size([68, 1, 256, 256])

In [22]:
import math

model.eval()
with torch.no_grad():
    test_x_tensor = torch.stack(test_x)
    test_y_tensor = torch.stack(test_y)
    
    residual_output = model(test_x_tensor)
    output = torch.stack(test_x) - residual_output
    
    loss = MSELoss(output, test_y_tensor)
    PSNR = 10 * math.log10(1 / loss.item())
    
print(f"PSNR: {PSNR}")

PSNR: 27.094637347853876
